In [13]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# 特征处理

In [17]:
df = df[~df.index.duplicated()]

In [ ]:
df = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df = pd.concat([df,df_add,df_test])
df = df[~df.index.duplicated()]
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['low_change'] = df.groupby(["ts_code"])['low'].pct_change()
df['high_change'] = df.groupby(["ts_code"])['high'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df['low_change'] = df['vol_change'].fillna(0)
df['high_change'] = df['vol_change'].fillna(0)
df = df[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
# l=0
# for arr in df['ts_code']:
#   df.loc[(df['ts_code'] ==arr) == True, '行业'] = l
#   l=l+1


In [ ]:
df = df[['pct_change','vol_change','high_change','low_change','y']]
df.head()

# LSTM

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

Using TensorFlow backend.


# 没有交叉验证


In [6]:
dataset = df.values
dataset

array([[ 1.33      ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.19      ,  0.03969754,  0.03969754,  0.03969754,  1.        ],
       [-0.39      , -0.13490358, -0.13490358, -0.13490358,  0.        ],
       ...,
       [-1.03      , -0.01741647, -0.01741647, -0.01741647,  0.        ],
       [ 2.87      ,  0.15638839,  0.15638839,  0.15638839,  1.        ],
       [ 3.88      ,  0.25827535,  0.25827535,  0.25827535,  1.        ]])

In [7]:
x = []
y = []
for i in range(len(dataset)-15):
    x.append(dataset[i:(i+10),:5].tolist())
    y.append(dataset[(i+10):(i+15),4,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1]*xtrain.shape[2], 1))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))
xtrain.shape


(31322, 10, 5)

In [9]:
regressor = Sequential()
regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (xtrain.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [10]:
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
31322/31322 [==============================] - 34s 1ms/step - loss: 0.2477
Epoch 2/100
31322/31322 [==============================] - 30s 971us/step - loss: 0.2454
Epoch 3/100
31322/31322 [==============================] - 33s 1ms/step - loss: 0.2417
Epoch 4/100
31322/31322 [==============================] - 33s 1ms/step - loss: 0.2373
Epoch 5/100
31322/31322 [==============================] - 29s 932us/step - loss: 0.2324
Epoch 6/100
31322/31322 [==============================] - 29s 923us/step - loss: 0.2272
Epoch 7/100
31322/31322 [==============================] - 28s 902us/step - loss: 0.2227
Epoch 8/100
31322/31322 [==============================] - 29s 922us/step - loss: 0.2180
Epoch 9/100
31322/31322 [==============================] - 30s 950us/step - loss: 0.2133
Epoch 10/100
31322/31322 [==============================] - 31s 998us/step - loss: 0.2084
Epoch 11/100
31322/31322 [==============================] - 30s 945

31322/31322 [==============================] - 28s 881us/step - loss: 0.0996
Epoch 90/100
31322/31322 [==============================] - 27s 878us/step - loss: 0.0993
Epoch 91/100
31322/31322 [==============================] - 28s 892us/step - loss: 0.0993
Epoch 92/100
31322/31322 [==============================] - 30s 949us/step - loss: 0.0984
Epoch 93/100
31322/31322 [==============================] - 28s 907us/step - loss: 0.0984
Epoch 94/100
31322/31322 [==============================] - 28s 905us/step - loss: 0.0976
Epoch 95/100
31322/31322 [==============================] - 28s 900us/step - loss: 0.0978
Epoch 96/100
31322/31322 [==============================] - 28s 899us/step - loss: 0.0971
Epoch 97/100
31322/31322 [==============================] - 28s 904us/step - loss: 0.0968
Epoch 98/100
31322/31322 [==============================] - 28s 907us/step - loss: 0.0963
Epoch 99/100
31322/31322 [==============================] - 28s 905us/step - loss: 0.0961
Epoch 100/100
31322/313

In [ ]:
from sklearn.externals import joblib
os.chdir("workspace/model_save")
oblib.dump(rf,'lstm.model')


In [11]:
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    each_a = auc(fpr, tpr)
    a.append(each_a)
a = pd.DataFrame(a)
a = a.dropna()
a.mean()

0    0.907831
dtype: float64

# 预测结果

In [25]:
df_test1= pd.read_csv(r'.././Data/20190506_STOCK.csv')
df_test2 = pd.read_csv(r'.././Data/20190514_STOCK.csv')
df_test3 = pd.read_csv(r'.././Data/20190521_STOCK.csv')
df_test4 = pd.concat([df_test1,df_test2,df_test3])
# df_test4 = df_test4[~df_test4.index.duplicated()]
df_test4
    


,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20190402,农林牧渔,3605.26,3485.11,3605.26,3513.06,-41.36,-1.16,236389.0,2761598.0,39.92,3.78,0
1,801020,20190402,采掘,2935.97,2905.01,2941.36,2935.37,22.18,0.76,282181.0,1732160.0,15.25,1.34,1
2,801030,20190402,化工,2971.25,2961.67,3011.61,2996.91,55.41,1.88,762659.0,7911222.0,18.94,2.06,1
3,801040,20190402,钢铁,2743.16,2711.59,2743.16,2732.47,0.80,0.03,195360.0,1062437.0,7.85,1.23,1
4,801050,20190402,有色金属,3195.75,3152.60,3261.40,3234.95,63.39,2.00,582889.0,5159532.0,32.45,2.26,1
5,801080,20190402,电子,2949.98,2924.79,2970.61,2935.14,0.62,0.02,683154.0,8117717.0,34.64,3.14,1
6,801110,20190402,家用电器,6970.77,6882.30,6973.44,6892.98,-70.05,-1.01,172337.0,1586252.0,15.45,3.13,0
7,801120,20190402,食品饮料,13082.51,12840.94,13082.51,12958.48,-151.07,-1.15,119683.0,2720395.0,31.06,5.83,0
8,801130,20190402,纺织服装,2322.71,2309.75,2332.17,2317.32,2.98,0.13,121569.0,941894.0,25.09,2.01,1
9,801140,20190402,轻工制造,2632.49,2610.28,2645.19,2620.00,6.92,0.26,251513.0,2633454.0,22.45,2.11,1


In [26]:
df_test4['vol_change'] = df_test4.groupby(["ts_code"])['vol'].pct_change()
df_test4['low_change'] = df_test4.groupby(["ts_code"])['low'].pct_change()
df_test4['high_change'] = df_test4.groupby(["ts_code"])['high'].pct_change()
df_test4 =  df_test4[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]


# 回测5.08-5.14得分

In [27]:
columns=['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']

In [28]:
df_test4.trade_date
index1=(df_test4.trade_date > 20190507)
df_huice_target = df_test4[index1]

In [29]:
index2= (df_test4.trade_date > 20190418)&(df_test4.trade_date < 20190508)
df_huice = df_test4[index2]

In [30]:
df_huice_target1=df_huice_target[(df_huice_target.trade_date==20190508)][['name']]
columns1=[20190508,20190509,20190510,20190513,20190514]
for arr1 in columns1:
    a=df_huice_target[(df_huice_target.trade_date==arr1)][['name','y']]
    a.columns=['name',arr1]
    df_huice_target1=df_huice_target1.merge(a,how='left',on='name')


In [38]:
df_huice_pr=pd.DataFrame()
df_huice_pr['trade_date']=[20190508,20190509,20190510,20190513,20190514]
for arr in columns:
    df4 = df_huice.loc[df_huice['name']==arr]
    dataset4=df4.values
    x4=dataset4[0:10,3:8].tolist()
    x4=np.array(x4)
    x4=np.reshape(x4, (1,x4.shape[0],x4.shape[1]))
    ytest4=regressor.predict(x4)
    y_p4=pd.DataFrame(ytest4)
    y_p4=y_p4.T
    y_p4['trade_date']=[20190508,20190509,20190510,20190513,20190514]
    y_p4.columns=[arr,'trade_date']
    df_huice_pr = df_huice_pr.merge(y_p4,how='left',on='trade_date')
df_huice_pr

,trade_date,农林牧渔,采掘,化工,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,...,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
0,20190508,0.830414,0.651229,0.886567,0.425479,0.921046,0.376896,0.056695,0.907593,0.916015,...,0.078055,0.930705,0.953214,0.966620,0.774992,0.193811,0.965737,0.430594,0.472301,0.845473
1,20190509,0.709021,0.078510,0.117813,0.212292,0.302713,0.285190,0.018692,0.482966,0.512121,...,0.641519,0.299541,0.740488,0.634303,0.879139,0.959044,0.968585,0.060365,0.126035,0.143408
2,20190510,0.109479,0.104185,0.627938,0.814101,0.974571,0.900695,0.112273,0.922782,0.935393,...,0.743395,0.937160,0.744136,0.066432,0.942935,0.923115,0.062989,0.424151,0.705852,0.848889
3,20190513,0.346984,0.014216,0.028945,0.195471,0.797934,0.429224,0.032033,0.160004,0.282427,...,0.015901,0.041544,0.986668,0.020246,0.917956,0.987214,0.231350,0.006851,0.026702,0.117916
4,20190514,0.978737,0.972971,0.929689,0.917033,0.951892,0.931973,0.958882,0.390148,0.889457,...,0.946787,0.955132,0.974071,0.958397,0.571688,0.835140,0.979206,0.952844,0.945218,0.935845


In [23]:
df_huice_pr.index=df_huice_pr['trade_date']
df_huice_pr=df_huice_pr.drop('trade_date',axis=1)

In [31]:
df_huice_target1

,name,20190508,20190509,20190510,20190513,20190514
0,农林牧渔,0,1,1,0,0
1,采掘,0,0,1,0,0
2,化工,0,0,1,0,0
3,钢铁,0,0,1,0,0
4,有色金属,1,0,1,0,1
5,电子,0,0,1,0,0
6,家用电器,0,0,1,0,0
7,食品饮料,0,0,1,1,0
8,纺织服装,1,0,1,1,0
9,轻工制造,1,0,1,0,0


In [41]:
toal=0
df_huice_pr1=df_huice_pr.T
df_huice_pr1.index=range(34)
for arr2 in df_huice_pr1.columns:
    f, t, t1 = roc_curve(df_huice_target1[arr2],df_huice_pr1[arr2],pos_label=1)
    toal=toal+auc(f, t)
    print(arr2,auc(f,t))
print('mean',toal/5)   

20190508 0.8551724137931035
20190509 0.6363636363636364
20190510 nan
20190513 0.484375
20190514 0.5698924731182795
mean nan


# 预测5.22-5.28

In [42]:
# 行业名字和行业代码的对应
df_me=pd.DataFrame(df_test4[['ts_code','name']])
df_me=df_me.drop_duplicates()
df_me
df8=df_me[['name']]
df8=df8.T
df8.values

array([['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']], dtype=object)

In [46]:
# 预测未来五天的结果

# colums=[801010, 801020, 801030, 801040, 801050, 801080, 801110, 801120,
#         801130, 801140, 801150, 801160, 801170, 801180, 801200, 801210,
#         801230, 801250, 801260, 801270, 801280, 801300, 801710, 801720,
#         801730, 801740, 801750, 801760, 801770, 801780, 801790, 801880,
#         801890, 802600]
range_index = (df_test4.trade_date > 20190507)
df_final = df_test4[range_index]
df_pr=pd.DataFrame()
for arr in columns:
    df3 = df_final.loc[df_final['name']==arr]
    dataset2=df3.values
    x1=dataset2[0:10,3:8].tolist()
    x1=np.array(x1)
    x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
    ytest1=regressor.predict(x1)
    y_p=pd.DataFrame(ytest1)
    y_p=y_p.T
    y_p['name']=arr
    y_p['trade_date']=[20190522,20190523,20190524,20190527,20190528]
    df_pr = pd.concat([df_pr,y_p])
df_pr=df_me.merge(df_pr,how='left',on='name')
df_pr.columns=['ts_code','name','p','trade_date']
df_pr=df_pr[['ts_code','trade_date','p']]
df_pr
# df_pr.to_csv('./结果提交.csv')

,ts_code,trade_date,p
0,801010,20190522,0.107383
1,801010,20190523,0.716985
2,801010,20190524,0.970274
3,801010,20190527,0.146391
4,801010,20190528,0.092339
5,801020,20190522,0.746949
6,801020,20190523,0.032162
7,801020,20190524,0.994971
8,801020,20190527,0.917691
9,801020,20190528,0.169424


# 交叉验证

In [47]:
kf = KFold(n_splits=10, shuffle=True, random_state=520)

In [ ]:
i=0
for train,test in kf.split(xtrain):
    a = []
    b= []
    X_train, X_valid= xtrain[train,:], xtrain[test,:]
    y_train, y_valid = ytrain[train,:], ytrain[test,:]
    print(i, 'fold')
    model=regressor.fit(X_train,y_train, epochs = 100, batch_size = 50)
    predicted_stock_price1 = regressor.predict(X_valid)
    for j in range(len(y_valid)):
        fpr1, tpr1, thresholds1 = roc_curve(y_valid[j],predicted_stock_price1[j], pos_label=1)
        a.append(auc(fpr1, tpr1))
    a=pd.DataFrame(a)
    a=a.dropna()
    print(a.mean())
    predicted_stock_price2 = regressor.predict(xtest)
    for k in range(len(ytest)):
        fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
        b.append(auc(fpr2, tpr2))
    b=pd.DataFrame(b)
    b=b.dropna()
    print(b.mean())
    i=i+1

0 fold
Epoch 1/100
28189/28189 [==============================] - 27s 973us/step - loss: 0.0953
Epoch 2/100
28189/28189 [==============================] - 27s 947us/step - loss: 0.0950
Epoch 3/100
28189/28189 [==============================] - 27s 948us/step - loss: 0.0946
Epoch 4/100
28189/28189 [==============================] - 27s 960us/step - loss: 0.0941
Epoch 5/100
28189/28189 [==============================] - 26s 938us/step - loss: 0.0937
Epoch 6/100
28189/28189 [==============================] - 27s 975us/step - loss: 0.0938
Epoch 7/100
28189/28189 [==============================] - 29s 1ms/step - loss: 0.0933
Epoch 8/100
28189/28189 [==============================] - 28s 980us/step - loss: 0.0931
Epoch 9/100
28189/28189 [==============================] - 27s 961us/step - loss: 0.0930
Epoch 10/100
28189/28189 [==============================] - 28s 980us/step - loss: 0.0915
Epoch 11/100
28189/28189 [==============================] - 27s 971us/step - loss: 0.0922
Epoch 12/100
2

28189/28189 [==============================] - 25s 902us/step - loss: 0.0704
Epoch 83/100
28189/28189 [==============================] - 25s 898us/step - loss: 0.0692
Epoch 84/100
28189/28189 [==============================] - 25s 899us/step - loss: 0.0696
Epoch 85/100
28189/28189 [==============================] - 25s 893us/step - loss: 0.0691
Epoch 86/100
28189/28189 [==============================] - 25s 897us/step - loss: 0.0695
Epoch 87/100
28189/28189 [==============================] - 25s 898us/step - loss: 0.0695
Epoch 88/100
28189/28189 [==============================] - 26s 924us/step - loss: 0.0690
Epoch 89/100
28189/28189 [==============================] - 28s 987us/step - loss: 0.0695
Epoch 90/100
28189/28189 [==============================] - 28s 981us/step - loss: 0.0694
Epoch 91/100
28189/28189 [==============================] - 28s 996us/step - loss: 0.0686
Epoch 92/100
28189/28189 [==============================] - 27s 967us/step - loss: 0.0685
Epoch 93/100
28189/2818

28190/28190 [==============================] - 25s 872us/step - loss: 0.0618
Epoch 63/100
28190/28190 [==============================] - 25s 878us/step - loss: 0.0611
Epoch 64/100
28190/28190 [==============================] - 25s 874us/step - loss: 0.0621
Epoch 65/100
28190/28190 [==============================] - 25s 870us/step - loss: 0.0623
Epoch 66/100
28190/28190 [==============================] - 25s 879us/step - loss: 0.0612
Epoch 67/100
28190/28190 [==============================] - 25s 877us/step - loss: 0.0613
Epoch 68/100
28190/28190 [==============================] - 29s 1ms/step - loss: 0.0609
Epoch 69/100
28190/28190 [==============================] - 27s 948us/step - loss: 0.0612
Epoch 70/100
28190/28190 [==============================] - 25s 881us/step - loss: 0.0612
Epoch 71/100
28190/28190 [==============================] - 25s 880us/step - loss: 0.0610
Epoch 72/100
28190/28190 [==============================] - 25s 877us/step - loss: 0.0610
Epoch 73/100
28190/28190 

28190/28190 [==============================] - 25s 883us/step - loss: 0.0568
Epoch 44/100
28190/28190 [==============================] - 25s 887us/step - loss: 0.0562
Epoch 45/100
28190/28190 [==============================] - 25s 879us/step - loss: 0.0565
Epoch 46/100
28190/28190 [==============================] - 25s 879us/step - loss: 0.0562
Epoch 47/100
28190/28190 [==============================] - 25s 883us/step - loss: 0.0559
Epoch 48/100
28190/28190 [==============================] - 25s 881us/step - loss: 0.0562
Epoch 49/100
28190/28190 [==============================] - 25s 884us/step - loss: 0.0564
Epoch 50/100
28190/28190 [==============================] - 27s 971us/step - loss: 0.0568
Epoch 51/100
28190/28190 [==============================] - 28s 1ms/step - loss: 0.0561
Epoch 52/100
28190/28190 [==============================] - 28s 997us/step - loss: 0.0556
Epoch 53/100
28190/28190 [==============================] - 28s 1000us/step - loss: 0.0565
Epoch 54/100
28190/28190

28190/28190 [==============================] - 25s 886us/step - loss: 0.0539
Epoch 25/100
28190/28190 [==============================] - 25s 885us/step - loss: 0.0543
Epoch 26/100
28190/28190 [==============================] - 25s 885us/step - loss: 0.0539
Epoch 27/100
28190/28190 [==============================] - 25s 887us/step - loss: 0.0540
Epoch 28/100
28190/28190 [==============================] - 25s 885us/step - loss: 0.0530
Epoch 29/100
28190/28190 [==============================] - 25s 882us/step - loss: 0.0530
Epoch 30/100
28190/28190 [==============================] - 25s 881us/step - loss: 0.0533
Epoch 31/100
28190/28190 [==============================] - 28s 992us/step - loss: 0.0534
Epoch 32/100
28190/28190 [==============================] - 27s 940us/step - loss: 0.0535
Epoch 33/100
28190/28190 [==============================] - 26s 932us/step - loss: 0.0536
Epoch 34/100
28190/28190 [==============================] - 26s 910us/step - loss: 0.0531
Epoch 35/100
28190/2819

# 新闻

In [0]:
df1 = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df1 = pd.concat([df1,df_add,df_test])

In [84]:
df2=df1[['name','ts_code']]
df2 = df2.drop_duplicates(keep='first')


,0,1152,2304,3456,3656,4608,5760,6912,8064,9216,10368,11520,12672,13824,14976,16128,17280,17480,18432,19584,20736,21888,23040,24192,25344,26496,27648,28800,29952,31104,32256,33408,34560,35712,36864,38016
name,农林牧渔,采掘,化工,黑色金属,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,轻工制造,医药生物,公用事业,交通运输,房地产,商业贸易,餐饮旅游,休闲服务,综合,申万制造,申万消费,申万投资,申万服务,申万300指数,建筑材料,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
ts_code,801010,801020,801030,801040,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600


In [86]:
df_news1 = pd.read_csv(r'./TRAINSET_NEWS.csv')
df_news2= pd.read_csv(r'./20190506_NEWS.csv')
df_news3= pd.read_csv(r'./20190514_NEWS.csv')
df_news = pd.concat([df_news1,df_news2,df_news3])
df_news.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27077 entries, 0 to 112
Data columns (total 4 columns):
id         27077 non-null object
date       27077 non-null int64
title      27048 non-null object
content    26915 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


# Tensorflow

In [0]:
import tensorflow as tf
time_step=10      #时间步
rnn_unit=25       #hidden layer units
batch_size=80     #每一批次训练多少个样例
input_size=3      #输入层维度
output_size=1     #输出层维度
lr=0.0006 


In [0]:
# 训练集
def get_train_data(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    batch_index=[]
    data_train=dataset[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
            x=normalized_train_data[i:i+time_step,:3]
            y=normalized_train_data[i:i+time_step,2,np.newaxis]
            train_x.append(x.tolist())
            train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y
 


In [0]:
# 测试集
def get_test_data(time_step=10,test_begin=30000):
    data_test=dataset[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:3]
        y=normalized_test_data[i*time_step:(i+1)*time_step,2]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:3]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,2]).tolist())
    return mean,std,test_x,test_y

In [0]:
# 模型搭建
weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [0]:
def train_lstm(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重复训练2000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))